# Keener and Sneyd Blood Model


In [27]:
import numpy as np
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.layouts import row, widgetbox, gridplot
from bokeh.models import CustomJS, Slider, Range1d, LabelSet, Label
import warnings

warnings.simplefilter('ignore')


def saturationFxn(oPP, kp, expTerm):
    percentSaturation = np.power(( oPP / (kp+oPP) ), expTerm) * 100
    return percentSaturation

oPP = np.arange(0,120,1)
#myoglobinPercentSaturation  = saturationFxn(oPP, kp_mg, expTerm_mg)
#hemoglobinPercentSaturation = saturationFxn(oPP, kp_hg, expTerm_hg)
kp_gen = 2.4
expTerm_gen = 1
genericPercentSaturation = saturationFxn(oPP, kp_gen, expTerm_gen)

source = ColumnDataSource(data=dict(oPP=oPP, genericPercentSaturation=genericPercentSaturation))


genericCasePlot = figure(title = "Keener & Sneyd Blood Model")
#genericCasePlot.xaxis.bounds = (0,np.max(oPP))
#genericCasePlot.yaxis.bounds = (0,100)
# make invisible points at minima and maxima to fix the plot
genericCasePlot.scatter(np.max(oPP),100,alpha=0)
genericCasePlot.scatter(0,0,alpha=0)
genericCasePlot.xaxis.axis_label = "Oxygen Partial Pressure (mm Hg)"
genericCasePlot.yaxis.axis_label = "Saturation (%)"

genericCasePlot.line('oPP','genericPercentSaturation', source = source, line_width = 3,line_alpha = 0.6)

kp_gen_slider = Slider(start=0, end= 50, value= 2.4, step= 0.1, title="Kp")
expTerm_gen_slider = Slider(start=0, end= 50, value= 1, step= 0.1,title="expTerm")

callback = CustomJS(args=dict(source=source, kp_gen = kp_gen_slider, exp_gen = expTerm_gen_slider), code="""
    var data = source.data;
    var oPP = data['oPP']
    var genericPercentSaturation = data['genericPercentSaturation']
    var kp = kp_gen.value
    var expTerm = exp_gen.value
    for (var indexVal = 0; indexVal < oPP.length; indexVal++) {
        genericPercentSaturation[indexVal] = (( oPP[indexVal] / (kp+oPP[indexVal]) ) ** expTerm) * 100;
    }
    source.change.emit();
""")

kp_gen_slider.js_on_change('value', callback)
expTerm_gen_slider.js_on_change('value', callback)



layout = row(
    genericCasePlot,
    widgetbox(kp_gen_slider, expTerm_gen_slider)
)

output_notebook()


show(layout)

Loading BokehJS ...